In [2]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, "../")

from autogluon.vision import ImagePredictor, ImageDataset
import numpy as np
import pandas as pd
import pickle
import datetime
from pathlib import Path
from sklearn.model_selection import StratifiedKFold
from cross_validation_autogluon import cross_val_predict_autogluon_image_dataset, train_predict_autogluon

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Read data

In [4]:
# change label index based on list of classes
def process_autogluon_image_dataset(dataset, classes):
    dataset["label_name"] = dataset.image.map(lambda x: Path(x).parts[-2])
    dataset["label"] = dataset["label_name"].map(lambda x: classes[x])
    return dataset

In [5]:
# original data deduped
DATA_PATH = "/Data/andrew-ng-dcai-comp-2021-data-deduped/andrew-ng-dcai-comp-2021-data/"

# read data from root folder
train_dataset, val_dataset, test_dataset = \
    ImageDataset.from_folders(
        root=DATA_PATH,
    )

# NOTE!
# NOTE! change the label index! AutoGluon assigns label index based on alphabetical order by default
classes = {
    label_name: idx
    for idx, label_name in enumerate(
        ["i", "ii", "iii", "iv", "v", "vi", "vii", "viii", "ix", "x"]
    )
}

train_dataset, val_dataset, test_dataset = \
    process_autogluon_image_dataset(train_dataset, classes), \
    process_autogluon_image_dataset(val_dataset, classes), \
    process_autogluon_image_dataset(test_dataset, classes)

# combine train and val dataset for cross-validation
train_val_dataset = pd.concat([train_dataset, val_dataset])

In [12]:
train_val_dataset.shape

(2831, 3)

In [16]:
train_val_dataset.groupby(["label", "label_name"])["image"].count()

label  label_name
0      i             335
1      ii            238
2      iii           256
3      iv            362
4      v             274
5      vi            263
6      vii           263
7      viii          282
8      ix            309
9      x             249
Name: image, dtype: int64

In [17]:
train_val_dataset.head()

,image,label,label_name
0,/Data/andrew-ng-dcai-comp-2021-data-deduped/andrew-ng-dcai-comp-2021-data/train/i/ab9fb784-ce5d-11eb-b317-38f9d35ea60f.png,0,i
1,/Data/andrew-ng-dcai-comp-2021-data-deduped/andrew-ng-dcai-comp-2021-data/train/i/aba24486-ce5d-11eb-b317-38f9d35ea60f.png,0,i
2,/Data/andrew-ng-dcai-comp-2021-data-deduped/andrew-ng-dcai-comp-2021-data/train/i/aba35128-ce5d-11eb-b317-38f9d35ea60f.png,0,i
3,/Data/andrew-ng-dcai-comp-2021-data-deduped/andrew-ng-dcai-comp-2021-data/train/i/aba4001e-ce5d-11eb-b317-38f9d35ea60f.png,0,i
4,/Data/andrew-ng-dcai-comp-2021-data-deduped/andrew-ng-dcai-comp-2021-data/train/i/aba4a622-ce5d-11eb-b317-38f9d35ea60f.png,0,i


## Generate Out-of-Sample Predicted Probabilities via Cross-Validation

In [19]:
%%time

# generate cross-validated predicted probabilities for various models so we can use them for ensemble scoring methods
models = [
    "resnet18",
    "resnet50d",
    "efficientnet_b1",
    "twins_pcpvt_base",
    "swin_base_patch4_window7_224"
]

epochs = 100
holdout_frac = 0.2
n_splits = 5

# run cross-validation for each model

ts = datetime.datetime.now().strftime("%Y%m%d%H%M%S") # CV start timestamp

for model in models:
    
    print("----")
    print(f"Running cross-validation for model: {model}")

    MODEL_PARAMS = {
        "model": model,
        "epochs": epochs,
        "holdout_frac": holdout_frac
    }

    # results of cross-validation will be saved to pickle files for each model/fold
    _ = \
        train_predict_autogluon(
            dataset=train_val_dataset, # train with NOISY LABELS
            classes=classes,
            out_folder=f"dcai_train_val_dataset_cv_{model}/", # save results of cross-validation in pickle files for each fold
            n_splits=n_splits,
            model_params=MODEL_PARAMS
        )

modified configs(<old> != <new>): {
root.img_cls.model   resnet101 != resnet18
root.train.early_stop_baseline 0.0 != -inf
root.train.batch_size 32 != 16
root.train.epochs    200 != 100
root.train.early_stop_max_value 1.0 != inf
root.train.early_stop_patience -1 != 10
root.misc.seed       42 != 84
root.misc.num_workers 4 != 16
}
Saved config to /dcai/src/experiments/andrew-ng-dcai/1cc2f720/.trial_0/config.yaml


----
Running cross-validation for model: resnet18
----
Running Cross-Validation on Split: 0


Model resnet18 created, param count:                                         11181642
AMP not enabled. Training in float32.
Disable EMA as it is not supported for now.
Start training from [Epoch 0]
Epoch[0] Batch [49]	Speed: 239.285470 samples/sec	accuracy=0.092500	lr=0.000100
Epoch[0] Batch [99]	Speed: 523.931972 samples/sec	accuracy=0.133125	lr=0.000100
[Epoch 0] training: accuracy=0.132874
[Epoch 0] speed: 353 samples/sec	time cost: 5.698930
[Epoch 0] validation: top1=0.149780 top5=0.709251
[Epoch 0] Current best top-1: 0.149780 vs previous -inf, saved to /dcai/src/experiments/andrew-ng-dcai/1cc2f720/.trial_0/best_checkpoint.pkl
Epoch[1] Batch [49]	Speed: 451.582510 samples/sec	accuracy=0.185000	lr=0.000250
Epoch[1] Batch [99]	Speed: 504.656697 samples/sec	accuracy=0.232500	lr=0.000250
[Epoch 1] training: accuracy=0.242618
[Epoch 1] speed: 490 samples/sec	time cost: 4.113294
[Epoch 1] validation: top1=0.339207 top5=0.837004
[Epoch 1] Current best top-1: 0.339207 vs previous 0.149780

Saving dcai_train_val_dataset_cv_resnet18_20220329175851/split_0/_test_pred_probs_split_0
Saving dcai_train_val_dataset_cv_resnet18_20220329175851/split_0/_test_pred_features_split_0


modified configs(<old> != <new>): {
root.img_cls.model   resnet101 != resnet18
root.train.early_stop_baseline 0.0 != -inf
root.train.batch_size 32 != 16
root.train.epochs    200 != 100
root.train.early_stop_max_value 1.0 != inf
root.train.early_stop_patience -1 != 10
root.misc.seed       42 != 344
root.misc.num_workers 4 != 16
}
Saved config to /dcai/src/experiments/andrew-ng-dcai/6f2c7aa5/.trial_0/config.yaml


...

Saving dcai_train_val_dataset_cv_swin_base_patch4_window7_224_20220329175851/split_2/_test_pred_probs_split_2
Saving dcai_train_val_dataset_cv_swin_base_patch4_window7_224_20220329175851/split_2/_test_pred_features_split_2
Saving dcai_train_val_dataset_cv_swin_base_patch4_window7_224_20220329175851/split_2/_test_labels_split_2
Saving dcai_train_val_dataset_cv_swin_base_patch4_window7_224_20220329175851/split_2/_test_image_files_split_2
Saving dcai_train_val_dataset_cv_swin_base_patch4_window7_224_20220329175851/split_2/_test_indices_split_2
----
Running Cross-Validation on Split: 3


Model swin_base_patch4_window7_224 created, param count:                                         86753474
AMP not enabled. Training in float32.
Disable EMA as it is not supported for now.
Start training from [Epoch 0]
Epoch[0] Batch [49]	Speed: 80.955239 samples/sec	accuracy=0.115000	lr=0.000100
Epoch[0] Batch [99]	Speed: 99.818224 samples/sec	accuracy=0.150000	lr=0.000100
[Epoch 0] training: accuracy=0.173720
[Epoch 0] speed: 90 samples/sec	time cost: 22.167973
[Epoch 0] validation: top1=0.317181 top5=0.757709
[Epoch 0] Current best top-1: 0.317181 vs previous -inf, saved to /dcai/src/experiments/andrew-ng-dcai/8d755c56/.trial_0/best_checkpoint.pkl
Epoch[1] Batch [49]	Speed: 98.761910 samples/sec	accuracy=0.275000	lr=0.000250
Epoch[1] Batch [99]	Speed: 100.769779 samples/sec	accuracy=0.329375	lr=0.000250
[Epoch 1] training: accuracy=0.353346
[Epoch 1] speed: 99 samples/sec	time cost: 20.296773
[Epoch 1] validation: top1=0.546256 top5=0.929515
[Epoch 1] Current best top-1: 0.546256 vs 

Saving dcai_train_val_dataset_cv_swin_base_patch4_window7_224_20220329175851/split_3/_test_pred_probs_split_3
Saving dcai_train_val_dataset_cv_swin_base_patch4_window7_224_20220329175851/split_3/_test_pred_features_split_3
Saving dcai_train_val_dataset_cv_swin_base_patch4_window7_224_20220329175851/split_3/_test_labels_split_3
Saving dcai_train_val_dataset_cv_swin_base_patch4_window7_224_20220329175851/split_3/_test_image_files_split_3
Saving dcai_train_val_dataset_cv_swin_base_patch4_window7_224_20220329175851/split_3/_test_indices_split_3
----
Running Cross-Validation on Split: 4


Model swin_base_patch4_window7_224 created, param count:                                         86753474
AMP not enabled. Training in float32.
Disable EMA as it is not supported for now.
Start training from [Epoch 0]
Epoch[0] Batch [49]	Speed: 82.739762 samples/sec	accuracy=0.150000	lr=0.000100
Epoch[0] Batch [99]	Speed: 99.684895 samples/sec	accuracy=0.171875	lr=0.000100
[Epoch 0] training: accuracy=0.189961
[Epoch 0] speed: 91 samples/sec	time cost: 21.959800
[Epoch 0] validation: top1=0.339207 top5=0.748899
[Epoch 0] Current best top-1: 0.339207 vs previous -inf, saved to /dcai/src/experiments/andrew-ng-dcai/1eb70707/.trial_0/best_checkpoint.pkl
Epoch[1] Batch [49]	Speed: 98.747726 samples/sec	accuracy=0.293750	lr=0.000250
Epoch[1] Batch [99]	Speed: 101.095218 samples/sec	accuracy=0.332500	lr=0.000250
[Epoch 1] training: accuracy=0.356299
[Epoch 1] speed: 99 samples/sec	time cost: 20.285553
[Epoch 1] validation: top1=0.511013 top5=0.938326
[Epoch 1] Current best top-1: 0.511013 vs 

Saving dcai_train_val_dataset_cv_swin_base_patch4_window7_224_20220329175851/split_4/_test_pred_probs_split_4
Saving dcai_train_val_dataset_cv_swin_base_patch4_window7_224_20220329175851/split_4/_test_pred_features_split_4
Saving dcai_train_val_dataset_cv_swin_base_patch4_window7_224_20220329175851/split_4/_test_labels_split_4
Saving dcai_train_val_dataset_cv_swin_base_patch4_window7_224_20220329175851/split_4/_test_image_files_split_4
Saving dcai_train_val_dataset_cv_swin_base_patch4_window7_224_20220329175851/split_4/_test_indices_split_4
CPU times: user 3h 45min 27s, sys: 17min 53s, total: 4h 3min 20s
Wall time: 4h 27min 38s


## Display GPU used for training

In [20]:
!nvidia-smi

Tue Mar 29 22:33:38 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.86       Driver Version: 470.86       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:05:00.0 Off |                  N/A |
| 41%   35C    P8     7W / 260W |    115MiB / 11011MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA RTX A6000    Off  | 00000000:09:00.0 Off |                  Off |
| 30%   